In [104]:
import pandas as pd
import numpy as np

from typing import Callable

In [105]:
data = pd.read_csv("train.csv")

targets = data.columns[-3:]

In [106]:
def window_sliding(data: pd.DataFrame, features: dict[str, list[tuple]], in_place=False):
    if not in_place:
        data = data.copy()
    
    for feature, functions in features.items():
        for func, sizes in functions.items():
            for w_size in sizes:
                rolling = data[feature].rolling(window=w_size, min_periods=1)
                data[f"{feature}_{func.__name__}_{w_size}h"] = rolling.agg(func)
    
    if not in_place:
        return data

In [107]:
data_cp = data.copy()

Зададим размеры окон в часах, основываясь на выделенных периодах. Воспользуемся функциями min, max, std. При необходимости можно изменить периоды или функции.

In [108]:
window_sliding(data_cp, dict(zip(list(targets), [{np.max: [24, 12, 8, 6], np.min: [24, 12, 8, 6], np.std: [24, 12, 8, 6]}, 
                                                           {np.max: [24, 12, 8, 6], np.min: [24, 12, 8, 6], np.std: [24, 12, 8, 6]}, 
                                                           {np.max: [24, 12, 8, 6], np.min: [24, 12, 8, 6], np.std: [24, 12, 8, 6]}])), in_place=True)

In [109]:
data_cp.head(5)

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,...,target_nitrogen_oxides_amax_8h,target_nitrogen_oxides_amax_6h,target_nitrogen_oxides_amin_24h,target_nitrogen_oxides_amin_12h,target_nitrogen_oxides_amin_8h,target_nitrogen_oxides_amin_6h,target_nitrogen_oxides_std_24h,target_nitrogen_oxides_std_12h,target_nitrogen_oxides_std_8h,target_nitrogen_oxides_std_6h
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,...,167.7,167.7,167.7,167.7,167.7,167.7,NaN,NaN,NaN,NaN
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,...,167.7,167.7,98.9,98.9,98.9,98.9,48.648947,48.648947,48.648947,48.648947
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,...,167.7,167.7,98.9,98.9,98.9,98.9,34.585739,34.585739,34.585739,34.585739
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,...,177.2,177.2,98.9,98.9,98.9,98.9,36.409923,36.409923,36.409923,36.409923
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,...,177.2,177.2,98.9,98.9,98.9,98.9,32.891230,32.891230,32.891230,32.891230


Также можно продифферецнировать ряд

In [110]:
def diff(targets_with_n: dict):
    for t, n in targets_with_n.items():
        data_cp[f'{t}_diff'] = data_cp[f'{t}'].diff(n)

In [111]:
diff(dict(zip(targets, [1, 1, 1])))

Также сделаем отдельные параметры для дня, месяца, года. Также сделаем отдельный параметр для сезонов, как это было сделано в EDA

In [112]:
data_cp.date_time = pd.to_datetime(data.date_time)

In [113]:
interval_1 = data.date_time < data.date_time[3450]
interval_2 = (data.date_time > data.date_time[3450]) & (data.date_time < data.date_time[4200])
interval_3 = data.date_time > data.date_time[4200]

data_cp["season"] = np.nan
data_cp.loc[interval_1, 'season'], data_cp.loc[interval_2, 'season'], data_cp.loc[interval_3, 'season'] = 1, 2, 3

In [115]:
data_cp["date_time_hour"] = data_cp.date_time.dt.hour
data_cp["date_time_day"] = data_cp.date_time.dt.day
data_cp["date_time_month"] = data_cp.date_time.dt.month
data_cp["date_time_year"] = data_cp.date_time.dt.year

In [121]:
data_cp.iloc[:, 9:].head(5)

,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,target_carbon_monoxide_amax_24h,target_carbon_monoxide_amax_12h,target_carbon_monoxide_amax_8h,target_carbon_monoxide_amax_6h,target_carbon_monoxide_amin_24h,target_carbon_monoxide_amin_12h,target_carbon_monoxide_amin_8h,...,target_nitrogen_oxides_std_8h,target_nitrogen_oxides_std_6h,target_carbon_monoxide_diff,target_benzene_diff,target_nitrogen_oxides_diff,season,date_time_hour,date_time_day,date_time_month,date_time_year
0,2.5,12.0,167.7,2.5,2.5,2.5,2.5,2.5,2.5,2.5,...,NaN,NaN,NaN,NaN,NaN,1.0,18,10,3,2010
1,2.1,9.9,98.9,2.5,2.5,2.5,2.5,2.1,2.1,2.1,...,48.648947,48.648947,-0.4,-2.1,-68.8,1.0,19,10,3,2010
2,2.2,9.2,127.1,2.5,2.5,2.5,2.5,2.1,2.1,2.1,...,34.585739,34.585739,0.1,-0.7,28.2,1.0,20,10,3,2010
3,2.2,9.7,177.2,2.5,2.5,2.5,2.5,2.1,2.1,2.1,...,36.409923,36.409923,0.0,0.5,50.1,1.0,21,10,3,2010
4,1.5,6.4,121.8,2.5,2.5,2.5,2.5,1.5,1.5,1.5,...,32.891230,32.891230,-0.7,-3.3,-55.4,1.0,22,10,3,2010


Также нам может понадобиться масштабирование признаков. Если этого не сделать, то в случае применения регуляризации модель будет работать хуже, так как у малых по модулю признаков, возможно, будут большие веса, а регуляризация "штрафует" за это. Также масштабирование повысит интерпретируемость, так как веса можно будет понимать как важность признака.

In [133]:
features = list(data_cp.columns[1:9]) + list(data_cp.columns[12:])

In [136]:
from sklearn.preprocessing import StandardScaler

X, Y = data_cp.dropna()[features], data_cp.dropna()[targets]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(X, Y)

In [147]:
features_scaled

array([[-9.67253964e-01, -1.30337634e-01, -9.64526719e-01, ...,
        -7.02354568e-01, -1.65860839e+00, -1.18619706e-02],
       [-1.04286561e+00,  4.96202312e-01, -9.02607069e-01, ...,
        -7.02354568e-01, -1.65860839e+00, -1.18619706e-02],
       [-1.24449667e+00,  8.52582831e-01, -8.11106372e-01, ...,
        -7.02354568e-01, -1.65860839e+00, -1.18619706e-02],
       ...,
       [-1.42092385e+00, -7.45381433e-01, -1.70279947e+00, ...,
         1.69599516e+00,  1.55327282e+00, -1.18619706e-02],
       [-1.62255492e+00, -3.94748987e-01, -1.75920400e+00, ...,
         1.69599516e+00,  1.55327282e+00, -1.18619706e-02],
       [-1.62255492e+00, -3.60260549e-01, -1.68650482e+00, ...,
        -1.73021874e+00, -2.37235977e+00,  8.43030249e+01]])